In [1]:
from models.userProfileModel import UserProfileModel
from models.popularityModel import PopularityModel
from models.targetModel import TargetModel
from data.loadData import *
from data.dataFunctions import get_played_songs_for_user_id, get_songs_by_traks_ids, get_played_tracks, find_random_n_track_ids
from sklearn.model_selection import train_test_split
from tqdm import tqdm

# Less track params for user profile model

In [2]:
TEST_SET = 0.3
N = 10

users = load_users()
tracks = load_tracks()
tracks_less = load_tracks_less()
artists = load_artists()

sessions = load_sessions()
sessions_train, sessions_test = train_test_split(
    sessions.values, test_size=TEST_SET
)
sessions_train = pd.DataFrame(
    data=sessions_train, columns=["user_id", "track_id", "event"]
)
sessions_test = pd.DataFrame(
    data=sessions_test, columns=["user_id", "track_id", "event"]
)

In [3]:
userProfilemodel = UserProfileModel()
userProfilemodel.fit(users, tracks, sessions_train)

userProfilemodel_less = UserProfileModel()
userProfilemodel_less.fit(users, tracks_less, sessions_train)

In [6]:
ALL_USERS = [
    [514, 281],
    [475, 386, 476],
    [317, 391],
    [299]
]

all_tracks_played = 0
avg_position, avg_position_less = 0, 0
in_best_n, in_best_n_less = 0, 0

for USERS in ALL_USERS:
    tracks_played_in_future = get_played_tracks(USERS, sessions_test)
    all_tracks_played += len(tracks_played_in_future)

    for examle_track_id in tqdm(tracks_played_in_future):

        evaluation_tracks = [examle_track_id]
        evaluation_tracks.extend(find_random_n_track_ids(100))

        ranked_songs = userProfilemodel.rank_tracks_for_users(
            USERS, evaluation_tracks
        )
        ranked_songs_less = userProfilemodel_less.rank_tracks_for_users(
            USERS, evaluation_tracks
        )

        avg_position += ranked_songs.index(examle_track_id)
        avg_position_less +=ranked_songs_less.index(examle_track_id)

        if ranked_songs.index(examle_track_id) < N:
            in_best_n += 1
        if ranked_songs_less.index(examle_track_id) < N:
            in_best_n_less += 1

print("================= All params =================")
print(
    "average position: ", round(avg_position / all_tracks_played, 2)
)
print(f"in top {N}: {in_best_n} times")
print(f"recall@{N}: {round(in_best_n/all_tracks_played * 100, 2)} %")

print("================= Less params =================")
print(
    "average position: ", round(avg_position_less / all_tracks_played, 2)
)
print(f"in top {N}: {in_best_n_less} times")
print(f"recall@{N}: {round(in_best_n_less/all_tracks_played * 100, 2)} %")


100%|██████████| 47/47 [02:00<00:00,  2.57s/it]

================= All params =================
average position:  49.91
in top 10: 114 times
recall@10: 10.63 %
================= Less params =================
average position:  51.29
in top 10: 110 times
recall@10: 10.26 %
